In [2]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

batch_size = 128
nb_classes = 10
nb_epoch = 20

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 64
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# generator for augmented training data
datagen = ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train)

# Define model
model = Sequential()

model.add(
    Convolution2D(
        nb_filters, 
        kernel_size[0], 
        kernel_size[1],
        border_mode='valid',
        input_shape=input_shape
    )
)
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Convolution2D(nb_filters*2, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters*2, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

model.fit_generator(
    datagen.flow(X_train, Y_train, batch_size=batch_size),
    samples_per_epoch=len(X_train),
    nb_epoch=nb_epoch,
    verbose=1,
    validation_data=(X_test, Y_test),
    callbacks=[EarlyStopping(patience=2, verbose=1)]
)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples
Epoch 1/20
60000/60000 [==============================] - 23s - loss: 0.3115 - acc: 0.8985 - val_loss: 0.0357 - val_acc: 0.9878
Epoch 2/20
60000/60000 [==============================] - 23s - loss: 0.0937 - acc: 0.9712 - val_loss: 0.0283 - val_acc: 0.9913
Epoch 3/20
60000/60000 [==============================] - 23s - loss: 0.0712 - acc: 0.9783 - val_loss: 0.0282 - val_acc: 0.9914
Epoch 4/20
60000/60000 [==============================] - 23s - loss: 0.0615 - acc: 0.9814 - val_loss: 0.0163 - val_acc: 0.9946
Epoch 5/20
60000/60000 [==============================] - 23s - loss: 0.0535 - acc: 0.9838 - val_loss: 0.0167 - val_acc: 0.9947
Epoch 6/20
60000/60000 [==============================] - 23s - loss: 0.0501 - acc: 0.9848 - val_loss: 0.0138 - val_acc: 0.9953
Epoch 7/20
60000/60000 [==============================] - 23s - loss: 0.0453 - acc: 0.9860 - val_loss: 0.0198 - val_acc: 0.9933
Epoch 8/20
60000/60000 [=======

In [ ]:
test = np.genfromtxt('test.csv', delimiter=',', skip_header=1)
print(test.shape)

test = test.reshape(test.shape[0], 1, img_rows, img_cols)
test /= 255
print(test.shape)

predictions = model.predict_classes(test)

labels = np.array(range(1, predictions.shape[0]+1))
results = np.column_stack([labels, predictions])

np.savetxt('result.csv', results, delimiter=',', header='ImageId,Label', fmt='%d', comments='')